In [209]:
import numpy as np
import time
import pandas as pd
import numpy.random as random
import matplotlib.pyplot as plt
import math
from gensim import corpora
from scipy import optimize
import scipy
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder


In [210]:
#load data
data = pd.read_csv('poliblogs2008.csv')

### Ingest corpus to create documents and vocab

In [5]:
# load corpus
documents = corpora.MmCorpus('corpus.mm')
dictionary = corpora.Dictionary.load('dictionary')

In [6]:
vocab = dictionary.token2id

In [7]:
def init_stm(documents, settings): 
      
    K = settings['dim']['K']
    V = settings['dim']['V']
    A = settings['dim']['A']
    N = settings['dim']['N']
    
    #Random initialization
    mu = np.array([0]*(K-1))[:,None]
    sigma = np.zeros(((K-1),(K-1)))
    diag = np.diagonal(sigma, 0)
    diag.setflags(write=True)
    diag.fill(20)
    beta = random.gamma(.1,1, V*K).reshape(K,V)
    beta = (beta / beta.sum(axis=1)[:,None])
    lambd = np.zeros((N, (K-1)))
    
    #turn beta into a list and assign it for each aspect
    beta = [beta, beta] # FOR A=2
    kappa_initialized = init_kappa(documents, K, V, A, interactions=settings['kappa']['interactions'])
      
    #create model object
    model = {'mu':mu, 'sigma':sigma, 'beta': beta, 'lambda': lambd, 'kappa':kappa_initialized}
    
    return(model)

def init_kappa(documents, K, V, A, interactions): 
    # read in documents and vocab
    flat_documents = [item for sublist in documents for item in sublist]
    m = []

    total_sum = sum(n for _, n in flat_documents)

    for elem in flat_documents: 
        m.append(elem[1] / total_sum)

    m = np.log(m) - np.log(np.mean(m)) #logit of m


    #Defining parameters
    aspectmod = A > 1 # if there is more than one topical content variable
    if(aspectmod):
        interact = interactions # allow for the choice to interact
    else:
        interact = FALSE

    #Create the parameters object
    parLength = K + A * aspectmod + (K*A)*interact

    #create covariates. one element per item in parameter list.
    #generation by type because its conceptually simpler
    if not aspectmod & interact:
        covar = {'k': np.arange(K),
             'a': np.repeat(np.nan, parLength), #why parLength? 
             'type': np.repeat(1, K)}

    if(aspectmod & interact == False):
        covar = {'k': np.append(np.arange(K), np.repeat(np.nan, A)),
                 'a': np.append(np.repeat(np.nan, K), np.arange(A)), 
                 'type': np.append(np.repeat(1, K), np.repeat(2, A))}      
    if(interact):
        covar = {'k': np.append(np.arange(K), np.append(np.repeat(np.nan, A), np.repeat(np.arange(K), A))),
                 'a': np.append(np.repeat(np.nan, K), np.append(np.arange(A), np.repeat(np.arange(A), K))), 
                 'type': np.append(np.repeat(1, K), np.append(np.repeat(2, A),  np.repeat(3,K*A)))}

    kappa = {'out': {'m':m,
                     'params' : np.tile(np.repeat(0,V), (parLength, 1)),
                     'covar' : covar
                     #'kappasum':, why rolling sum?
                    }
            }

    return(kappa['out'])

In [8]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def softmax_weights(x, weight):
    """Compute softmax values for each sets of scores in x."""
    e_x = weight*np.exp(x - np.max(x))[:,None]
    return e_x / e_x.sum(axis=0)

def lhood(eta, mu, siginv, doc_ct, Ndoc, eta_long, beta_tuple, phi, theta, neta):
    
    #formula 
    #rewrite LSE to prevent overflow
    part1 = np.sum(doc_ct * (eta_long.max() + np.log(np.exp(eta_long - eta_long.max())@beta_tuple)))-np.sum(doc_ct)*scipy.special.logsumexp(eta)
    part2 = .5*(eta-mu)@siginv@(eta-mu)
    
    out = part2 - part1
    
    return -out

def grad(eta, mu, siginv, doc_ct,  Ndoc, eta_long, beta_tuple, phi, theta, neta):

    #formula
    part1 = np.delete(np.sum(phi * doc_ct,axis=1) - np.sum(doc_ct)*theta, neta)
    part2 = siginv@(eta-mu)

    return part2 - part1

In [9]:
def e_step(documents, mu, sigma, lambd, beta):
    #quickly define useful constants
    V = beta['beta'][0].shape[1] # ncol
    K = beta['beta'][0].shape[0] # nrow
    N = len(documents)
    A = len(beta['beta'])
    
    # 1) Initialize Sufficient Statistics 
    sigma_ss = np.zeros(((K-1),(K-1)))
    if A == 2: 
        beta_ss = [np.zeros((K,V)), np.zeros((K,V))]
    else:
        print('Error: Only two metadata columns allowed.')
    bound = np.repeat(0,N)
    #lambd = np.repeat(0,N)
    
    # 2) Precalculate common components
    sigobj = np.linalg.cholesky(sigma) #initialization of sigma not positive definite
    sigmaentropy = np.sum(np.log(np.diag(sigobj)))
    siginv = np.linalg.inv(sigobj).T*np.linalg.inv(sigobj)
    
    # 3) Document Scheduling
    # For right now we are just doing everything in serial.
    # the challenge with multicore is efficient scheduling while
    # maintaining a small dimension for the sufficient statistics.
    
    mu = mu.flatten()
    
    #set parameters for one document (i)
    for i in range(N):

        eta=lambd[i]
        neta = len(eta)
        eta_long = np.insert(eta,-1,0)

        doc = documents[i]
        words = [x for x,y in doc]
        aspect = betaindex[i]
        #beta_i = beta['beta'][aspect][:,[words]] # replace with beta_ss[aspect][:,np.array(words)]
        beta_tuple = beta['beta'][aspect][:,np.array(words)]

        #set document specs
        doc_ct = np.array([y for x,y in doc]) #count of words in document
        Ndoc = np.sum(doc_ct)

        # initial values
        #beta_tuple = beta_i.reshape(K,beta_i.shape[2])
        theta = softmax(eta_long)
        phi = softmax_weights(eta_long, beta_tuple)
        # optimize variational posterior
        result = optimize.fmin_bfgs(lhood,x0=eta,
                           args=(mu, siginv, Ndoc, doc_ct, eta_long, beta_tuple, phi, theta, neta),
                           fprime=grad)
        #solve hpb
        doc_results = hpb(eta=result,
                          doc_ct=doc_ct,
                          mu=mu,
                          siginv=siginv,
                          beta_tuple=beta_tuple,
                          sigmaentropy=sigmaentropy,
                          theta=theta)
        
        # update sufficient statistics        
        #print(f"Input:eta: {doc_results['eta'].get('nu').shape}\nphi:{doc_results['phi'].shape}")
        print(f"\nbound:{doc_results['bound']}")
        sigma_ss = sigma_ss + doc_results['eta'].get('nu')


        #beta_ss[aspect][:,[words]] = beta_ss[aspect][:,[words]].reshape(K,beta_ss[aspect][:,[words]].shape[2]) 
        
        beta_ss[aspect][:,np.array(words)] = doc_results.get('phi') + np.take(beta_ss[aspect], words, 1)
        bound[i] = doc_results.get('bound')
        lambd[i] = doc_results['eta'].get('lambd')
        
        #4) Combine and Return Sufficient Statistics
        results = {'sigma':sigma_ss, 'beta':beta_ss, 'bound': bound, 'lambd': lambd}
        
    return results

# Solve for Hessian/Phi/Bound returning the result

In [ ]:
def hpb(eta, doc_ct, mu,siginv, beta_tuple,sigmaentropy, theta):
    #print(f'Input for the Hessian Phi Bound\neta:{eta.shape}\ndoc_ct:{doc_ct.shape}\nmu:{mu.shape}\nsiginv:{siginv.shape}\nbeta_tuple:{beta_tuple.shape}\nsigmaentropy:{sigmaentropy.shape}\ntheta:{theta.shape}')
    eta_long = np.insert(eta,-1,0)
    # copy to mess with 
    beta_temp = beta_tuple
    #column-wise multiplication of beta and expeta
    beta_temp = beta_temp*eta_long[:,None]
    
    beta_temp = (np.sqrt(doc_ct)[:,None] / np.sum(beta_temp, axis=0)[:,None]) * beta_temp.T # with shape (VxK)
    hess = beta_temp.T@beta_temp-np.sum(doc_ct)*(theta*theta.T) # hessian with shape KxK
    #print(f'hess.shape:{hess.shape}')
    #we don't need beta_temp any more so we turn it into phi 
    beta_temp = beta_temp.T * np.sqrt(doc_ct) # should equal phi ?! 

    np.fill_diagonal(hess, np.diag(hess)-np.sum(beta_temp, axis=1)-np.sum(doc_ct)*theta) #altered diagonal of h
    
    # drop last row and columns
    hess = np.delete(hess,eta.size,0)
    hess = np.delete(hess,eta.size,1)
    hess = hess + siginv # at this point, the hessian is complete

    # Invert hessian via cholesky decomposition 
    #np.linalg.cholesky(hess)
    # error -> not properly converged: make the matrix positive definite
    
    dvec = hess.diagonal()
    magnitudes = sum(abs(hess), 1) - abs(dvec)
    # cholesky decomposition works only for symmetric and positive definite matrices
    dvec = np.where(dvec < magnitudes, magnitudes, dvec)
    # A Hermitian diagonally dominant matrix A with real non-negative diagonal entries is positive semidefinite. 
    np.fill_diagonal(hess, dvec)
    #that was sufficient to ensure positive definiteness so no we can do cholesky 
    nu = np.linalg.cholesky(hess)

    #compute 1/2 the determinant from the cholesky decomposition
    detTerm = -np.sum(np.log(nu.diagonal()))
    #print(f'detTerm {detTerm}')

    #Finish constructing nu
    nu = np.linalg.inv(np.triu(nu))
    nu = nu@nu.T
    # precompute the difference since we use it twice
    diff = eta-mu
    #print(f'diff (shape):{diff, diff.shape}')
    ############## generate the bound and make it a scalar ##################
    #print('Compute the lower bound...')
    #print(f'shape of theta[:,None]: {theta[None,:].shape}')
    #print(f'shape of beta_temp: {beta_temp.shape}')
    #print(f'shape of the first summand: {(np.log(theta[None:,]@beta_temp)[None:,]@doc_ct).shape}')
    #print(f'first summand: {np.log(theta[None:,]@beta_temp)@doc_ct}')
    #print(f'shape of the second summand: {(detTerm - .5*diff.T@siginv@diff).shape}')
    #print(f'second summand: {detTerm - .5*diff.T@siginv@diff}')
    bound = np.log(theta[None:,]@beta_temp)@doc_ct + detTerm - .5*diff.T@siginv@diff - sigmaentropy 
    ###################### return values as dictionary ######################
    phi = beta_temp
    eta = {'lambd' : eta, 'nu':nu}
    
    result = {'phi':phi,'eta': eta,'bound': bound}
    
    return result

In [10]:
prevalence = 'blog'
content = 'rating'
num_topics = 10

# Setting control variables

In [192]:
def makeTopMatrix(x, data=None):
    return(data.loc[:,x]) # add intercept! 

xmat = makeTopMatrix(content, data)
yvar = xmat.astype('category')
yvarlevels = set(yvar)
betaindex = yvar.cat.codes
A = len(set(betaindex))

interactions = True #settings.kappa
verbose = True

init_type = "Random" #settings.init
ngroups = 1 #settings.ngroups
max_em_its = 15 #settings.convergence
emtol = 1e-5 #settings.convergence

#gamma_prior=("Pooled","L1") # settings.gamma.prior
#sigma_prior=0 #settings.sigma.prior
#kappa_prior=("L1","Jeffreys") # settings.kappa.prior

#Initialize parameters

settings = {
    'dim':{
        'K': num_topics, #number of topics
        'V' : len(dictionary), #number of words
        'A' : A, #dimension of topical content
        'N' : len(documents),
    },
    'kappa':{
        'interactions':True,
        'fixedintercept': True,
        'contrats': False,
        'mstep': {'tol':0.01, 'maxit':5}},
    'tau':{
        'mode': np.nan,
        'tol': 1e-5,
        'enet':1,
        'nlambda':250,
        'lambda.min.ratio':.001,
        'ic.k':2,
        'maxit':1e4},
    'init':{
        'mode':init_type, 
        'nits':20,
        'burnin':25,
        'alpha':50/num_topics,
        'eta':.01,
        's':.05,
        'p':3000},
    'convergence':{
        'max.em.its':max_em_its,
        'em.converge.thresh':emtol,
        'allow.neg.change':True,},
    'covariates':{
        'X':xmat,
        'betaindex':betaindex,
        'yvarlevels':yvarlevels,
        'formula': prevalence,},
    'gamma':{
        'mode':'L1', #needs to be set for the m-step (update mu in the topical prevalence model)
        'prior':np.nan,
        'enet':1, 
        'ic.k':2,
        'maxits':1000,},
    'sigma':{
        #'prior':sigma_prior,
        'ngroups':ngroups,},
}

In [214]:
def stm_control(documents, vocab, settings, model=None):
    
    ##########
    #Step 1: Initialize Parameters
    ##########
    
    #ngroups = settings$ngroups
    
    if model == None:
        print('Call init_stm()')
        model = init_stm(documents, settings) #initialize
    else: 
        model = model
        
    # unpack initialized model
    
    mu = model['mu']
    sigma = model['sigma']
    lambd = model['lambda'] 
    beta = {'beta': model['beta'],
            'kappa': model['kappa']}
    
    convergence = None
    
    #discard the old object
    del model
    
    betaindex = settings['covariates']['betaindex']
    
    #Pull out some book keeping elements
    betaindex = settings['covariates']['betaindex']
    
    ############
    #Step 2: Run EM
    ############
    
    t1 = time.process_time()

    #run the model (so far: one iteration)
    #suffstats = [] 

    ############
    # Run E-Step

    suffstats = (e_step(documents, mu, sigma, lambd, beta))
    
    sigma_ss = suffstats.get('sigma')
    lambd = suffstats.get('lambd')
    beta_ss = suffstats.get('beta')
    bound_ss = suffstats.get('bound')
    
    print(lambd.shape)
    
    ############
    # Run M-Step 
    
    # mu = opt_mu(lambd,
    #  covar=settings['covariates']['X'],
    #   enet=settings['gamma']['enet'],
    #   ic_k=settings['gamma']['ic.k'],
    #   maxits = settings['gamma']['maxits'],
    #   mode = settings['gamma']['mode'])
       

    
    print("Completed E-Step ({} seconds). \n".format(math.floor((time.process_time()-t1))))

    return lambd, beta_ss, sigma_ss, bound_ss


From **A Model of Text for Experimentation in the Social Sciences (Roberts et al.)**:
- $\beta_k$ is a V-dimensional probability mass function that controls the frequency according to which terms are generated from that topic.

From **The Structural Topic Model and Applied Social Science (Roberts et al.)**:
- 'For the nonconjugate logistic normal variables in the E-step we use a Laplace approximation.'

## E-Step

### Likelihood Function (for the nonconjugate variable) 

$f(\hat{\eta}_{d}) \propto  - \frac{1}{2} (\eta_d-\mu_d)^T \sum^{-1}(\eta_d-\mu_d)+\big(\sum_v c_{d,v} log\sum_k \beta_{k,v} e^{\eta_{d,k}}- W_d log \sum_k e^{\eta_{d,k}}\big)$

### Gradient of the Likelihood

$\nabla f(\eta_d)_k = (\sum c_{d,v} \langle \phi_{d,v,k} \rangle) - W_d\theta_{d,k}-\big(\sum^{-1}(\eta_d-\mu_d)\big)_k$

with $\theta_{d,k} = \frac{exp(\eta)}{\sum exp(\eta)}$


and $\langle\phi_{d,k,v}\rangle = \frac{exp(\eta_{d,k}) \beta_{d,v,k}}{\sum_k exp(\eta_{d,k}) \beta_{d,v,k}} $

### Variational posterior: 
$q(\eta_d) \sim N(\hat{\eta}_d, -\nabla^2 f(\hat{\eta}_d)^{-1})$

In [215]:
#e-step iteration for all N documents: 52 seconds 

out = stm_control(documents, vocab, settings)

Call init_stm()
Optimization terminated successfully.
         Current function value: -424530791.782004
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14

bound:-415160099.2223
Optimization terminated successfully.
         Current function value: -1773677809.268783
         Iterations: 4
         Function evaluations: 13
         Gradient evaluations: 13

bound:-1746424290.9161155
Optimization terminated successfully.
         Current function value: -466810439.186824
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14

bound:-454960501.48076206
Optimization terminated successfully.
         Current function value: -601007804.405791
         Iterations: 6
         Function evaluations: 15
         Gradient evaluations: 15

bound:-589461270.0252227
Optimization terminated successfully.
         Current function value: -412916272.726461
         Iterations: 4
         Function evaluations: 13
         Gradi

# Sigmaentropy

- Logarithmic sum of the diagonals of the variance-covariance matrix of topics -> integer ?!
- ```np.sum(np.log(np.diag(sigobj))```, where ```sigobj``` is the lower diagonal of the cholesky factor L
    - c.f. https://numpy.org/doc/stable/reference/generated/numpy.linalg.cholesky.html

## M-Step


Perform a penalized regression on $\lambda_k$ to get the updates for $\gamma_k$ based on the prevalence covariates (family: "mgaussian"). The prior on document-topic proportions maximizes the approximate ELBO with respect to the document-specific mean $\mu_{d,k} = X_d \gamma_k$ and the topic covariance matrix $\Sigma$.

Updates for $\gamma_k$ correspond to linear regression for each topic under the user specified prior with $\lambda_k$ as the outcome variable.

**NOTE**: Cross-Validation for the Penalized Regression?!

### Topical Prevalence Model Update

In [201]:
def opt_mu(lambd, covar, enet, ic_k, maxits, mode = "L1"):
    #prepare covariate matrix for modeling 
    covar = covar.astype('category')
    covar2D = covar[:,None] #prepares 1D array for one-hot encoding by making it 2D
    #create and fit OHE
    enc = OneHotEncoder(handle_unknown='ignore') 
    covarOHE = enc.fit_transform(covar2D).toarray()
    
    # TO-DO: mode = CTM if there are no covariates 
    # TO-DO: mode = Pooled if there are covariates requires variational linear regression with Half-Cauchy hyperprior
    
    # mode = L1 simplest method requires only glmnet (https://cran.r-project.org/web/packages/glmnet/index.html)
    if mode == "L1":
        model = linear_model.Lasso(alpha=enet)
        fitted_model = model.fit(covarOHE,lambd)
    else: 
        raise ValueError('Optimizing the parameter mu requires a mode. Choose from "CTM", "Pooled" or "L1" (default).')

    gamma = np.insert(fitted_model.coef_, 0, fitted_model.intercept_)
    # insert an intercept column into the covariate matrix
    design_matrix = np.c_[ np.ones(covarOHE.shape[0]), covarOHE]
    #compute mu as a linear combination of X and gamma
    print(gamma.shape)
    print(design_matrix.shape)
    mu = design_matrix@gamma

    return {
        #'mu':mu,
        'gamma':gamma
        }

In [219]:
lambd = out[0]

In [222]:
lambd.shape
# 

(13246, 9)

In [223]:
opt_mu(
    lambd[0],
    covar=settings['covariates']['X'],
    enet=settings['gamma']['enet'],
    ic_k=settings['gamma']['ic.k'],
    maxits = settings['gamma']['maxits'],
    mode = settings['gamma']['mode']
    )

/var/folders/mj/mqt0qs7x0njf2_k3_mr5dcg80000gn/T/ipykernel_39814/3769805511.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  covar2D = covar[:,None] #prepares 1D array for one-hot encoding by making it 2D


ValueError: Found input variables with inconsistent numbers of samples: [13246, 9]

In [199]:
covar = covar.astype('category')
covar2D = covar[:,None] #prepares 1D array for one-hot encoding by making it 2D
enc = OneHotEncoder(handle_unknown='ignore') #create OHE
covarOHE = enc.fit_transform(covar2D).toarray() #fit OHE
design_matrix = np.c_[ np.ones(covarOHE.shape[0]), covarOHE]
test = design_matrix@gamma

/var/folders/mj/mqt0qs7x0njf2_k3_mr5dcg80000gn/T/ipykernel_39814/1733353521.py:2: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  covar2D = covar[:,None] #prepares 1D array for one-hot encoding by making it 2D


In [ ]:
def opt_sigma():
    pass

### Topical Content Model Update

In [34]:
def opt_beta(beta_ss, kappa, settings):
    #if its standard lda just row normalize
    if kappa is None: 
        return list(beta=list(beta_ss[[1]]/np.sum(beta_ss[[1]])))
    #if its a SAGE model (Eisenstein et al., 2013) use the distributed poissons
    # if settings['tau']['mode'] == "L1":
    #     out = mnreg(beta_ss, settings) 
    # else: 
    #     out = jeffreysKappa(beta_ss, kappa, settings)
    # return out


### Global Covariance Update